**Code 2 -
Create a Machine Reading Comprehension and Calculate Evaluation Metrics -
Mohamadreza Jafaei -
400131017 ***

In [ ]:
!pip install datasets transformers datasets huggingface_hub
!apt install git-lfs
!pip install hazm

In [ ]:
# !wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
# !wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-stanford.zip
# !unzip resources-0.5.zip -d resources
# !unzip resources-stanford.zip -d resources

In [ ]:
import transformers
from transformers.utils import send_example_telemetry
from datasets import load_dataset, load_metric
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import torch
import numpy as np
import collections
from tqdm.auto import tqdm

## ||| SELECT MODEL

In [ ]:
model_names = [
    "bert-base-cased", # 0
    "prajjwal1/bert-tiny", # 1
    "prajjwal1/bert-small", # 2
    "distilbert-base-uncased", # 3
    "albert-base-v2", # 4
    "xlm-roberta-base", # 5
    "HooshvareLab/bert-fa-zwnj-base", # 6
    "HooshvareLab/distilbert-fa-zwnj-base", # 7
    "HooshvareLab/roberta-fa-zwnj-base", # 8
]
model_id = 5 # 7
model_checkpoint = model_names[model_id]

batch_size = 16

In [ ]:
datasets = load_dataset("gholamreza/pquad")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/63994 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/e49d5f650d69a5999fe6ceb4438a023cccdcf3e6519abc4dabce736f91595591


Generating validation split:   0%|          | 0/7976 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/ea42ddfa9db6f39bc3249a878c853a6f6b466f6217a360bbb8afbac9410d84cc


Generating test split:   0%|          | 0/8002 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/d6ba3b80ff2a6d0333454fac286694b5e777518ea141e0dcd7c0558b71624882
Dataset pquad downloaded and prepared to /root/.cache/huggingface/datasets/gholamreza___pquad/pquad/1.0.0/ce0e1bf6a7a67398a195ccb3a16263336e977abad6e67800390346e8d9530ab0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(f"Size of Train dataset: {len(datasets['train'])}")
print(f"Size of Valid dataset: {len(datasets['validation'])}")
print(f"Size of Test dataset:  {len(datasets['test'])}")

Size of Train dataset: 63994
Size of Valid dataset: 7976
Size of Test dataset:  8002


In [ ]:
def create_search_query(text):
    normalizer = Normalizer()
    lemmatizer = Lemmatizer()
    tagger = POSTagger(model='resources/postagger.model')
    chunker = Chunker(model='resources/chunker.model')
    # normalize = normalizer.normalize(text)
    words = word_tokenize(text)
    pos_tags = tagger.tag(words)
    tree = chunker.parse(pos_tags)
    query_words = []
    for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
        for word, pos in subtree.leaves():
            query_words.append(word)#(lemmatizer.lemmatize(word))
    query = ' '.join(query_words)
    return query

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer("اسم شما چیست؟ اسم من محمدرضا جفائی است.")

{'input_ids': [0, 20162, 2768, 50905, 1245, 20162, 230, 213732, 4569, 106057, 140, 477, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max_length = 384
doc_stride = 128

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/63994 [00:00<?, ? examples/s]

Map:   0%|          | 0/7976 [00:00<?, ? examples/s]

Map:   0%|          | 0/8002 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:

args = TrainingArguments(
    f"xlm-roberta-base-on-pquad-dataset",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 64043
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12009
  Number of trainable parameters = 75179522


Epoch,Training Loss,Validation Loss
1,1.136500,1.141361
2,0.853900,1.086229
3,0.646300,1.139201


Saving model checkpoint to distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-500
Configuration saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-500/config.json
Model weights saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-500/special_tokens_map.json
Saving model checkpoint to distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-1000
Configuration saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-1000/config.json
Model weights saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-1000/tokenizer_config.json
Special tokens file saved in distilbert-fa-zwnj-base-on-pquad-dataset/checkpoint-1000/special_tokens_map.json
Saving model chec

TrainOutput(global_step=12009, training_loss=0.9915697071333691, metrics={'train_runtime': 7225.1692, 'train_samples_per_second': 26.592, 'train_steps_per_second': 1.662, 'total_flos': 1.8826687092036096e+16, 'train_loss': 0.9915697071333691, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainer.save_model("/content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset")

Saving model checkpoint to /content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset
Configuration saved in /content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset/config.json
Model weights saved in /content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset/special_tokens_map.json


## Evaluation

In [ ]:
# target_dataset = "validation"
target_dataset = "test"

In [ ]:
# trainer.model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/CN/distilbert-fa-zwnj-base-on-pquad-dataset")

In [ ]:
device = torch.device("cpu")
trainer.model.to(device)

In [ ]:
n_best_size = 20

In [ ]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = datasets[target_dataset].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets[target_dataset].column_names
)

Map:   0%|          | 0/8002 [00:00<?, ? examples/s]

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
max_answer_length = 30

In [ ]:
squad_v2 = True

In [ ]:

examples = datasets[target_dataset]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(datasets[target_dataset], validation_features, raw_predictions.predictions)

Post-processing 8002 example predictions split into 8026 features.


  0%|          | 0/8002 [00:00<?, ?it/s]

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

<ipython-input-45-69495f532713>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


## ||| RESULTS

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets[target_dataset]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 66.52086978255436,
 'f1': 80.32762100115654,
 'total': 8002,
 'HasAns_exact': 60.16754270696452,
 'HasAns_f1': 78.31498410828735,
 'HasAns_total': 6088,
 'NoAns_exact': 86.72936259143155,
 'NoAns_f1': 86.72936259143155,
 'NoAns_total': 1914,
 'best_exact': 66.52086978255436,
 'best_exact_thresh': 0.0,
 'best_f1': 80.3276210011566,
 'best_f1_thresh': 0.0}